# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [138]:
from collections import OrderedDict
import functools
class LruCache(object):
    def __init__(self, max_count):
        self.items = OrderedDict()
        self.max_count = max_count
        
    def __getitem__(self, key):
        return self.items[key]
    
    def __setitem__(self, key, value):
        self.items[key] = value
    
    def __len__(self):
        return len(self.items)

#### Реализуйте декоратор

In [139]:
def cached(max_count):
    cache = LruCache(max_count)
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args = tuple(args)
            kwargs_keys = tuple(kwargs.keys())
            kwargs_values = tuple(kwargs.values())
            key = args + kwargs_keys + kwargs_values 
            if key in cache.items.keys():
                print('{} is in cache!'.format(key))
                return cache[key]
            
            else:
                value = func(*args, **kwargs)
                cache[key] = value
                if len(cache) > cache.max_count:
                    cache.items.popitem(last=False)
                return value
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [140]:
@cached(3)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(2)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

print("Call fact(8) =", fact(8), ". Now 6,7,8 are in cache.")
print("Call fact(7) =", fact(7))
print("Call fact(6) =", fact(6))
print("Call fact(5) =", fact(5), "is not in cache!")
print("Call fact1(8) =", fact1(8))
print("Call fact1(7) =", fact1(7))
print("Call fact1(6) =", fact1(6), "is not in cache!")
print("Different functions have different caches.")

Call fact(8) = 40320 . Now 6,7,8 are in cache.
(7,) is in cache!
Call fact(7) = 5040
(6,) is in cache!
Call fact(6) = 720
Call fact(5) = 120 is not in cache!
Call fact1(8) = 40320
(7,) is in cache!
Call fact1(7) = 5040
Call fact1(6) = 720 is not in cache!
Different functions have different caches.


#### Сравните свою реализацию с lru_cache из functools

Для этого сравним честное время работы LruCache(без print'ов) и functools.lru_cache:

In [141]:
def cached(max_count):
    cache = LruCache(max_count)
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args = tuple(args)
            kwargs_keys = tuple(kwargs.keys())
            kwargs_values = tuple(kwargs.values())
            key = args + kwargs_keys + kwargs_values 
            if key in cache.items.keys():
                #print('{} is in cache!'.format(key))
                return cache[key]
            
            else:
                value = func(*args, **kwargs)
                cache[key] = value
                if len(cache) > cache.max_count:
                    cache.items.popitem(last=False)
                return value
        return wrapper
    return deprecated

In [142]:
import time
from functools import lru_cache
lru_cache_start_time = time.time()

@lru_cache(5)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

fact(20)
fact(19)
fact(18)
fact(14)

lru_cache_end_time = time.time()
print('@lru_cache works {}'.format(lru_cache_end_time - lru_cache_start_time))

@lru_cache works 0.06985735893249512


In [143]:
cached_start_time = time.time()
@cached(5)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

fact(20)
fact(19)
fact(18)
fact(14)

cached_end_time = time.time()
print('@cache works {}'.format(cached_end_time - cached_start_time))

@cache works 0.04704904556274414


### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [144]:
def checked(*types):
    def deprecated(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if len(args) > len(types):
                raise TypeError("Too many arguments!")
            for arg, argtype in zip(args, types):
                if arg is not None and not isinstance(arg, argtype):
                    raise TypeError('Type of parameter {} is not {}'.format(arg, argtype.__name__))
            return func(*args, **kwargs)
        return wrapper
    return deprecated

#### Проверьте использование декоратора

In [145]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [146]:
strange_func(1,2,[1])

TypeError: Type of parameter 1 is not str

In [147]:
strange_func(1,2,3,4)

TypeError: Too many arguments!

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

Воспользуемся вспомогательной функцией sigleton для декорирования класса Logger:

In [148]:
import sys
import functools

def singleton(cls):
    instance = None
    @functools.wraps(cls)
    def wrapper(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    return wrapper

In [149]:
import sys

@singleton
class Logger(object):
    
    def __init__(self):
        self.index = 0
    
    def log_function(self, func, file, *args, **kwargs):
        result = func(*args, **kwargs)
        if file == sys.stdout or sys.stderr:
            print(self.index, datetime.datetime.now(), func.__name__, args, kwargs,result)
        else:
            f = open(file, 'w')
            f.write('{} {} {} {} {} {}'.format(self.index, str(now), func.__name__, args, kwargs, result))
            f.close()
        self.index += 1
        return
    
    def __call__(self, func):
        logger = Logger()
        def deprecated(func):
            @functools.wraps(func)
            def wrapper(*args, **kwargs):
                return logger.log_function(func, file, *args, **kwargs)
            return wrapper
        return deprecated

In [150]:
@Logger(sys.stderr)
def test1():
    return 'I am testing function 1'

@Logger(sys.stdout)
def test2():
    return 'I am testing function 2'

TypeError: __init__() takes 1 positional argument but 2 were given